In [1]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

print(f"This is the repo dir {repo_dir}")

This is the repo dir /Users/kwasia/Documents/Projects/ai-starter-kit


In [2]:
import pandas as pd
from utils.eval.rag_eval import (
    RAGEvaluator,
    RAGEvalConfig,
    load_pipeline,
    load_eval_dataframe,
)
from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load config
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

In [ ]:
# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

In [ ]:
# Use Case 1: CSV file with pre-generated answers, no context
eval_df = pd.read_csv("../data/res.csv")
results1 = evaluator.evaluate(eval_df)
print("Results 1:", results1)

In [ ]:
# Use Case 2: CSV file with pre-generated answers and context, add context_col to eval_dataset yaml section
# Re- Load config after changes
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)


results2 = evaluator.evaluate(eval_df)
print("Results 2:", results2)

In [ ]:
# Use Case 3: CSV file without answers, generate with pipelines, no context
# Re- Load config after changes
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    print("conf:", conf)
    llm_name, llm_config = config.get_llm_config(conf)
    print("llm_name:", llm_name)
    print("llm_config:", llm_config)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))

print("eval_llms:", eval_llms)

eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

eval_df = pd.read_csv("../data/res.csv")
pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [config.get_llm_config(conf) for conf in config.llm_configs]
]
results3 = evaluator.evaluate(eval_df, pipelines)
print("Results 3:", results3)

In [3]:
# Use Case 4: CSV file without answers, generate with pipelines, with context from vector DB
# Re- Load config after changes
config_path = "../../utils/eval/config.yaml"
config = RAGEvalConfig(config_yaml_path=config_path)

# Create evaluator
eval_llms = []
for conf in config.eval_llm_configs:
    llm_name, llm_config = config.get_llm_config(conf)
    eval_llm = SambaStudio(**llm_config)
    eval_llms.append((llm_name, eval_llm))


eval_embeddings = HuggingFaceInstructEmbeddings(model_name=config.embedding_model_name)
evaluator = RAGEvaluator(eval_llms, eval_embeddings, config_path)

eval_df = pd.read_csv("../data/res.csv")
pipelines = [
    load_pipeline((llm_name, SambaStudio(**llm_config)), config)
    for llm_name, llm_config in [config.get_llm_config(conf) for conf in config.llm_configs]
]
results4 = evaluator.evaluate(eval_df, pipelines)
print("Results 4:", results4)

/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


load INSTRUCTOR_Transformer


/Users/kwasia/.pyenv/versions/3.11.3/envs/adi-rag-eval/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512
This is llm instance SambaStudio
Params: {'model_kwargs': {'select_expert': 'Meta-Llama-3-70B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}}
load INSTRUCTOR_Transformer
max_seq_length  512
This is the vector db ../data/adi_vdb_v3_md_200.chromadb
This is the vector db ../data/adi_vdb_v3_md_200.chromadb
This is llm instance SambaStudio
Params: {'model_kwargs': {'select_expert': 'Meta-Llama-3-8B-Instruct', 'process_prompt': False, 'max_tokens_to_generate': 512}}
load INSTRUCTOR_Transformer
max_seq_length  512
This is the vector db ../data/adi_vdb_v3_md_200.chromadb
This is the vector db ../data/adi_vdb_v3_md_200.chromadb
[Document(page_content='adp1046\n\nTable 128. Register 0x82-STORE_USER_ALL\n| Bits | Bit Name | Type | Description |\n| :--- | :--- | :--- | :--- |\n| N/A | STORE_USER_ALL | Send <br> byte | Copy the entire contents of operating memory (registers) into EEPROM (Page 1 of the main <br> block). The EEPROM must first be unlocked. |', m

Evaluating: 100%|██████████| 3/3 [00:11<00:00,  3.80s/it]


...

[{'question': 'What is the maximum efficiency of the ADP2107?', 'answer': "\n        I'm happy to help you with your question! However, I must point out that the provided context does not mention the ADP2107, which is a specific IC. The context appears to be related to the ADP1046, ADP1055, and other registers.\n\nIf you meant to ask about the ADP1046, I can try to help you with that. However, I don't see any information about the maximum efficiency of the ADP1046 in the provided context. The context seems to be focused on registers and their descriptions, rather than performance characteristics.\n\nIf you could provide more context or clarify which IC you are referring to, I'll do my best to assist you.assistant\nassistant\nI'm happy to help you with your question! However, I must point out that the provided context does not mention the ADP2107, which is a specific IC. The context appears to be related to the ADP1046, ADP1055, and other registers.\n\nIf you meant to ask about the ADP1

Evaluating: 100%|██████████| 3/3 [00:11<00:00,  3.84s/it]


[{'question': 'What is the maximum efficiency of the ADP2107?', 'answer': "\n        I'm happy to help you with your question! However, I must point out that the provided context does not mention the ADP2107, which is a specific IC. The context appears to be related to the ADP1046, ADP1055, and other registers.\n\nIf you meant to ask about the ADP1046, I can try to help you with that. However, I don't see any information about the maximum efficiency of the ADP1046 in the provided context. The context seems to be focused on registers and their descriptions, rather than performance characteristics.\n\nIf you could provide more context or clarify which IC you are referring to, I'll do my best to assist you.assistant\nassistant\nI'm happy to help you with your question! However, I must point out that the provided context does not mention the ADP2107, which is a specific IC. The context appears to be related to the ADP1046, ADP1055, and other registers.\n\nIf you meant to ask about the ADP1

Evaluating: 100%|██████████| 3/3 [00:42<00:00, 14.12s/it]
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: kwasi-ankomah (ai-solutions-team). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current proces

Results 4: {'LLAMA70B_GEN_EVAL_LLAMA38B': {'answer_relevancy': 0.0000, 'answer_correctness': 0.9397, 'answer_similarity': 0.7589}, 'LLAMA70B_GEN_EVAL_LLAMA370B': {'answer_relevancy': 0.0000, 'answer_correctness': 0.1897, 'answer_similarity': 0.7589}, 'LLAMA8B_GEN_EVAL_LLAMA38B': {'answer_relevancy': 0.0000, 'answer_correctness': 0.1888, 'answer_similarity': 0.7552}, 'LLAMA8B_GEN_EVAL_LLAMA370B': {'answer_relevancy': 0.0000, 'answer_correctness': 0.1888, 'answer_similarity': 0.7552}}


In [ ]:
# Use Case 5: Evaluate on HF dataset
config.config["eval_dataset"]["hf_dataset_name"] = "squad_v2"
eval_df = load_eval_dataframe(config)
pipelines = [
    load_pipeline(
        SambaStudio(
            sambastudio_base_url=conf["base_url"],
            sambastudio_project_id=conf["project_id"],
            sambastudio_endpoint_id=conf["endpoint_id"],
            sambastudio_api_key=conf["api_key"],
            **conf["model_kwargs"]
        ),
        config,
    )
    for conf in config.llm_configs
]
results5 = evaluator.evaluate(eval_df, pipelines)
print("Results 5:", results5)